In [11]:
import pandas as pd
import os

# web crawling packages
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

from lxml import html
import urllib.request
from html_table_parser.parser import HTMLTableParser
from pprint import pprint

# helper functions
from crawling_help import game_stat_parser

# expoand pandas dataframe to not cut off text
pd.set_option('display.max_colwidth', None)

# Table of Contents
- [Gather List of Game Ids to Crawl](#attempting-to-use-NFL-game-stat-logic-to-use-for-NCAA)
- [Game ID Crawling](#Crawl-these-weekly-scoreboard-urls-for-game-id)
- [Game Stat Crawling](#Game-Stat-Crawling)

In [2]:
# college football data
odds_df = pd.read_csv('data/Package Sample 20240511/ncaaf_game_scores_1g_2022_sample.csv')
odds_df.head()

,season,date,away_team,away_score,away_point_spread,away_point_spread_line,away_money_line,home_team,home_score,home_point_spread,home_point_spread_line,home_money_line,over_under,over_line,under_line
0,2022-23,2022-08-27,austin_peay,27,27.5,-112,0.0,western_kentucky,38,-27.5,-112,0.0,64.5,-109,-109
1,2022-23,2022-08-27,northwestern,31,11.0,-110,350.0,nebraska,28,-11.0,-110,-439.0,52.0,-110,-110
2,2022-23,2022-08-27,wyoming,6,14.0,-110,430.0,illinois,38,-14.0,-110,-550.0,43.0,-110,-110
3,2022-23,2022-08-27,idaho_state,21,23.0,-112,0.0,unlv,52,-23.0,-112,0.0,52.5,-109,-109
4,2022-23,2022-08-27,sf_austin,17,-6.5,-125,0.0,jacksonville_state,42,6.5,-105,0.0,51.5,-112,-112


need someway to get a game_id for this odds data. That way I can add features around game statistics

## **attempting to use NFL game stat logic to use for NCAA**

In [18]:
## espn group number for each conference
conference_dict = {
    'FBS (I-A)': 80,
    'ACC': 1,
    'American': 151,
    'Big 12': 4,
    'Big Ten': 5,
    'CUSA': 12,
    'FBS Independents': 18,
    'MAC': 15,
    'Mountain West': 17,
    'Pac-12': 9,
    'SEC': 8,
    'Sun Belt': 37,
    'FBS (I-AA)': 81,
    'Big Sky': 20,
    'Big South-OVC': 179,
    'CAA': 48,
    'FCS Independents': 32,
    'Ivy': 22,
    'MEAC': 24,
    'MVFC': 21,
    'NEC': 25,
    'Patriot': 27,
    'Pioneer': 28,
    'SWAC': 31,
    'Southern': 29,
    'Southland': 30,
    'UAC': 177
}

In [22]:
# for conference in conference_dict.keys():
#     print(conference)
#     print(conference_dict[conference])

In [44]:
url_list = []

# year_list = ['2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
# year_list = ['2023']
year_list = ['2022']

for i in year_list:
    year = str(i)

    for i in range(1,16):
        week = str(i)

        # iterate through each conference
        for conference in conference_dict.keys():
            conference_group_var = conference_dict[conference]
            url_var = f'https://www.espn.com/college-football/scoreboard/_/week/{week}/year/{year}/seasontype/2/group/{conference_group_var}'
            url_list.append([year, week, conference_group_var, url_var])

url_df = pd.DataFrame(url_list, columns = ['year', 'week', 'conference', 'url'])
print(url_df.shape)
url_df.head()

(405, 4)


,year,week,conference,url
0,2022,1,80,https://www.espn.com/college-football/scoreboard/_/week/1/year/2022/seasontype/2/group/80
1,2022,1,1,https://www.espn.com/college-football/scoreboard/_/week/1/year/2022/seasontype/2/group/1
2,2022,1,151,https://www.espn.com/college-football/scoreboard/_/week/1/year/2022/seasontype/2/group/151
3,2022,1,4,https://www.espn.com/college-football/scoreboard/_/week/1/year/2022/seasontype/2/group/4
4,2022,1,5,https://www.espn.com/college-football/scoreboard/_/week/1/year/2022/seasontype/2/group/5


In [45]:
url_df.iloc[0]['url']

'https://www.espn.com/college-football/scoreboard/_/week/1/year/2022/seasontype/2/group/80'

## Crawl these weekly scoreboard urls for game id

### test

### default just shows the top 25 ranked games

In [4]:
driver_service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=driver_service)
# driver.get('https://www.espn.com/college-football/scoreboard/_/week/14/year/2023/seasontype/2')
driver.get('https://www.espn.com/college-football/scoreboard/_/week/2/year/2023/seasontype/2')

first_day = driver.find_elements(By.CSS_SELECTOR, 'section.Scoreboard.bg-clr-white.flex')

In [7]:
len(first_day)

23

In [10]:
# for i in first_day:
#     # print(i.text)
#     print(i.get_attribute("id"))

In [19]:
driver_service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=driver_service)
# driver.get('https://www.espn.com/college-football/scoreboard/_/week/14/year/2023/seasontype/2')
driver.get('https://www.espn.com/college-football/matchup/_/gameId/401525469')

game_date = driver.find_elements(By.CLASS_NAME, 'n8 GameInfo__Meta')

In [21]:
print(len(game_date))
for i in game_date:
    print(i.text)

0


In [ ]:
<div class="n8 GameInfo__Meta"><span>6:30 PM, September 7, 2023</span><span>Coverage: ACCN</span></div>

### schedule page - didnt end up going this route

In [24]:
# # driver_service = Service(ChromeDriverManager().install())
# # driver = webdriver.Chrome(service=driver_service)
# # driver.get('https://www.espn.com/college-football/scoreboard/_/week/14/year/2023/seasontype/2')
# driver.get('https://www.espn.com/college-football/schedule/_/week/2/year/2023/seasontype/2')

# games = driver.find_elements(By.CSS_SELECTOR, 'td.teams__col')

In [17]:
# print(len(games))

# for i in games:
#     print(i.text)
#     print(i.get_attribute("href"))

### loop

In [51]:
game_id_df = pd.DataFrame(columns = ['id', 'url', 'away_rank', 'away_team_name', 'away_record', 'home_rank', 'home_team_name', 'home_record'])
# Set up the Chrome WebDriver
driver_service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=driver_service)

## full df
for url in url_df['url']:

# # test with first 5 rows
# for url in url_df['url'][:5]:

    # set up the driver
    driver.get(url)
    
    try:
        game_elements = driver.find_elements(By.CSS_SELECTOR, 'section.Scoreboard.bg-clr-white.flex')

        game_list = []
        for p in range(len(game_elements)):
            game_list.append(game_elements[p].get_attribute("id"))
        df_tmp = pd.DataFrame(game_list, columns = ['id'])

    except Exception as e:
        print("Error Couldn't Find First element:", e)

    try:
        away_team_names = driver.find_elements(By.CSS_SELECTOR, 'li.ScoreboardScoreCell__Item--away')

        away_team_vars = pd.DataFrame(columns = ['away_rank', 'away_team_name', 'away_record'])

        for away_team_elem in away_team_names:
            # print(i.text.split('\n')[0])

            # test if i.text.split('\n')[0] is a number, if so keep that and name this rank_var
            if away_team_elem.text.split('\n')[0].isdigit():
                rank_var = away_team_elem.text.split('\n')[0]
                team_name_var = away_team_elem.text.split('\n')[1]
                record_var = away_team_elem.text.split('\n')[2]
            else:
                rank_var = ''
                team_name_var = away_team_elem.text.split('\n')[0]
                record_var = away_team_elem.text.split('\n')[1]

            away_team_vars_tmp = pd.DataFrame({'away_rank': [rank_var], 'away_team_name': [team_name_var], 'away_record': [record_var]})
            away_team_vars = pd.concat([away_team_vars, away_team_vars_tmp], axis = 0)
            away_team_vars = away_team_vars.reset_index(drop = True)

    except Exception as e:
        print("Error Couldn't Find Away Team Names:", e)

    try:
        home_team_names = driver.find_elements(By.CSS_SELECTOR, 'li.ScoreboardScoreCell__Item--home')

        home_team_vars = pd.DataFrame(columns = ['home_rank', 'home_team_name', 'home_record'])
        
        for home_team_elem in home_team_names:
            # print(i.text.split('\n')[0])

            # test if i.text.split('\n')[0] is a number, if so keep that and name this rank_var
            if home_team_elem.text.split('\n')[0].isdigit():
                rank_var = home_team_elem.text.split('\n')[0]
                team_name_var = home_team_elem.text.split('\n')[1]
                record_var = home_team_elem.text.split('\n')[2]
            else:
                rank_var = ''
                team_name_var = home_team_elem.text.split('\n')[0]
                record_var = home_team_elem.text.split('\n')[1]

            home_team_vars_tmp = pd.DataFrame({'home_rank': [rank_var], 'home_team_name': [team_name_var], 'home_record': [record_var]})
            home_team_vars = pd.concat([home_team_vars, home_team_vars_tmp], axis = 0)
            home_team_vars = home_team_vars.reset_index(drop = True)
            
    except Exception as e:
        print("Error Couldn't Find Home Team Names:", e)

    df_tmp['url'] = url
    df_tmp = pd.concat([df_tmp, away_team_vars, home_team_vars], axis = 1)

    game_id_df = pd.concat([game_id_df, df_tmp], axis = 0)
    game_id_df = game_id_df.reset_index(drop = True)

maybe could be useful to wrap into a function...not sure if when we are live if this will be the way we collect game ids though...1 perk here is that we would get the rank and record from here.

In [52]:
game_id_df

,id,url,away_rank,away_team_name,away_record,home_rank,home_team_name,home_record
0,401426532,https://www.espn.com/college-football/scoreboard/_/week/1/year/2022/seasontype/2/group/80,,Austin Peay,(0-10-1Away),,Western Kentucky,(1-01-0Home)
1,401405059,https://www.espn.com/college-football/scoreboard/_/week/1/year/2022/seasontype/2/group/80,,Nebraska,(0-10-1Big Ten),,Northwestern,(1-01-0Big Ten)
2,401413256,https://www.espn.com/college-football/scoreboard/_/week/1/year/2022/seasontype/2/group/80,,Idaho State,(0-10-1Away),,UNLV,(1-01-0Home)
3,401404146,https://www.espn.com/college-football/scoreboard/_/week/1/year/2022/seasontype/2/group/80,,UConn,(0-10-1Away),,Utah State,(1-01-0Home)
4,401405058,https://www.espn.com/college-football/scoreboard/_/week/1/year/2022/seasontype/2/group/80,,Wyoming,(0-10-1Away),,Illinois,(1-01-0Home)
...,...,...,...,...,...,...,...,...
3520,401507650,https://www.espn.com/college-football/scoreboard/_/week/15/year/2022/seasontype/2/group/21,,Samford,(11-25-2Away),,North Dakota State,(11-27-1Home)
3521,401507651,https://www.espn.com/college-football/scoreboard/_/week/15/year/2022/seasontype/2/group/21,,Holy Cross,(12-16-1Away),,South Dakota State,(12-18-0Home)
3522,401507651,https://www.espn.com/college-football/scoreboard/_/week/15/year/2022/seasontype/2/group/27,,Holy Cross,(12-16-1Away),,South Dakota State,(12-18-0Home)
3523,401507650,https://www.espn.com/college-football/scoreboard/_/week/15/year/2022/seasontype/2/group/29,,Samford,(11-25-2Away),,North Dakota State,(11-27-1Home)


In [54]:
game_id_df[game_id_df['id'].duplicated()].sort_values(by = 'id')

,id,url,away_rank,away_team_name,away_record,home_rank,home_team_name,home_record
181,401403853,https://www.espn.com/college-football/scoreboard/_/week/1/year/2022/seasontype/2/group/17,,Vanderbilt,(1-01-0Away),,Hawai'i,(0-10-1Home)
205,401403853,https://www.espn.com/college-football/scoreboard/_/week/1/year/2022/seasontype/2/group/8,,Vanderbilt,(1-01-0Away),,Hawai'i,(0-10-1Home)
184,401403854,https://www.espn.com/college-football/scoreboard/_/week/1/year/2022/seasontype/2/group/17,,Utah State,(1-10-1Away),1,Alabama,(1-01-0Home)
208,401403854,https://www.espn.com/college-football/scoreboard/_/week/1/year/2022/seasontype/2/group/8,,Utah State,(1-10-1Away),1,Alabama,(1-01-0Home)
212,401403855,https://www.espn.com/college-football/scoreboard/_/week/1/year/2022/seasontype/2/group/8,23,Cincinnati,(0-10-1Away),19,Arkansas,(1-01-0Home)
...,...,...,...,...,...,...,...,...
3522,401507651,https://www.espn.com/college-football/scoreboard/_/week/15/year/2022/seasontype/2/group/27,,Holy Cross,(12-16-1Away),,South Dakota State,(12-18-0Home)
3518,401507652,https://www.espn.com/college-football/scoreboard/_/week/15/year/2022/seasontype/2/group/20,,Incarnate Word,(12-16-1Away),,Sacramento State,(12-16-1Home)
3524,401507652,https://www.espn.com/college-football/scoreboard/_/week/15/year/2022/seasontype/2/group/30,,Incarnate Word,(12-16-1Away),,Sacramento State,(12-16-1Home)
3519,401507670,https://www.espn.com/college-football/scoreboard/_/week/15/year/2022/seasontype/2/group/48,,William & Mary,(11-26-1Away),,Montana State,(12-18-0Home)


In [55]:
game_id_df[game_id_df['id'] == '401403853']

,id,url,away_rank,away_team_name,away_record,home_rank,home_team_name,home_record
10,401403853,https://www.espn.com/college-football/scoreboard/_/week/1/year/2022/seasontype/2/group/80,,Vanderbilt,(1-01-0Away),,Hawai'i,(0-10-1Home)
181,401403853,https://www.espn.com/college-football/scoreboard/_/week/1/year/2022/seasontype/2/group/17,,Vanderbilt,(1-01-0Away),,Hawai'i,(0-10-1Home)
205,401403853,https://www.espn.com/college-football/scoreboard/_/week/1/year/2022/seasontype/2/group/8,,Vanderbilt,(1-01-0Away),,Hawai'i,(0-10-1Home)


## when we crawled this we sometimes get duplicated if this isnt a conference game because it shows up in both teams conference filter

In [56]:
# check for duplciates using id column
print("Total Games Scraped", game_id_df.shape)

print("Games Duplicated", game_id_df['id'].duplicated().sum())

game_id_df_no_dups = game_id_df.drop_duplicates(subset = 'id')

print("After dropping dups", game_id_df_no_dups.shape)

# save file to the following path - data/data_crawling/table_games/2023/games.csv
games_file_path = 'data/data_crawling/table_games/2022/games.csv'

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(games_file_path), exist_ok=True)

game_id_df_no_dups.to_csv(games_file_path, index = False)

Total Games Scraped (3525, 8)
Games Duplicated 1969
After dropping dups (1556, 8)


## Game Stat Crawling

### test 1 game

In [57]:
# Opens a website and read its
# binary contents (HTTP Response Body)
def url_get_contents(url):

    # Opens a website and read its
    # binary contents (HTTP Response Body)

    #making request to the website
    req = urllib.request.Request(url=url)
    f = urllib.request.urlopen(req)

    #reading contents of the website
    return f.read()

In [ ]:
# defining the html contents of a URL.
xhtml = url_get_contents('https://www.espn.com/college-football/matchup/_/gameId/401525469').decode('utf-8')

# Defining the HTMLTableParser object
p = HTMLTableParser()

# feeding the html contents in the
# HTMLTableParser object
p.feed(xhtml)

# Now finally obtaining the data of
# the table required
#pprint(p.tables[1])

df_testing = pd.DataFrame(p.tables[1])
column_list = list(df_testing[0].values)

In [74]:
# defining the html contents of a URL.
xhtml = url_get_contents('https://www.espn.com/college-football/matchup/_/gameId/401525469').decode('utf-8')

# Defining the HTMLTableParser object
p = HTMLTableParser()

# feeding the html contents in the
# HTMLTableParser object
p.feed(xhtml)

# Now finally obtaining the data of
# the table required
#pprint(p.tables[1])

df_testing = pd.DataFrame(p.tables[1])
column_list = list(df_testing[0].values)

away = pd.DataFrame(df_testing[1]).T
away.columns = column_list
away = away.reset_index()
# away = away.drop(columns = ['index', 'Matchup'])
away = away.drop(columns = ['index'])
#away.drop(away.columns[[13]], axis=1, inplace=True)
away = away.rename(columns = {'1st Downs': 'first_downs_away',
                            # 'Passing 1st downs': 'first_downs_passing_away',
                            # 'Rushing 1st downs': 'first_downs_rushing_away', 
                            # '1st downs from penalties': 'first_down_by_penalty_away', 
                            '3rd down efficiency': 'third_down_away', 
                            '4th down efficiency': 'fourth_down_away', 
                            # 'Total Plays': 'total_plays_away', 
                            'Total Yards': 'total_yards_away', 
                            # 'Total Drives': 'total_drives_away', 
                            # 'Yards per Play': 'yards_per_play_away', 
                            'Passing': 'passing_away', 
                            'Comp-Att': 'comp_att_away', 
                            'Yards per pass': 'yards_per_pass_away',  
                            # 'Sacks-Yards Lost': 'sack_yards_lost_away', 
                            'Rushing': 'rushing_away', 
                            'Rushing Attempts': 'rushing_att_away', 
                            'Yards per rush': 'yards_per_rush_away', 
                            # 'Red Zone (Made-Att)': 'red_zone_att_away', 
                            'Penalties': 'penalty_away', 
                            'Turnovers': 'turnovers_away', 
                            'Interceptions thrown': 'int_thrown_away',
                            'Fumbles lost': 'fumbles_lost_away',
                            # 'Rushing 1st downs': 'first_down_rushing_away', 
                            # 'Defensive / Special Teams TDs': 'defensive_td_away', 
                            'Possession': 'possession_away'})

home = pd.DataFrame(df_testing[2]).T
home.columns = column_list
home = home.reset_index()
# home = home.drop(columns = ['index', 'Matchup'])
home = home.drop(columns = ['index'])
#home.drop(home.columns[[13]], axis=1, inplace=True)
home = home.rename(columns = {'1st Downs': 'first_downs_home',
                            # 'Passing 1st downs': 'first_downs_passing_home',
                            # 'Rushing 1st downs': 'first_downs_rushing_home', 
                            # '1st downs from penalties': 'first_down_by_penalty_home', 
                            '3rd down efficiency': 'third_down_home', 
                            '4th down efficiency': 'fourth_down_home', 
                            # 'Total Plays': 'total_plays_home', 
                            'Total Yards': 'total_yards_home', 
                            # 'Total Drives': 'total_drives_home', 
                            # 'Yards per Play': 'yards_per_play_home', 
                            'Passing': 'passing_home', 
                            'Comp-Att': 'comp_att_home', 
                            'Yards per pass': 'yards_per_pass_home',  
                            # 'Sacks-Yards Lost': 'sack_yards_lost_home', 
                            'Rushing': 'rushing_home', 
                            'Rushing Attempts': 'rushing_att_home', 
                            'Yards per rush': 'yards_per_rush_home', 
                            # 'Red Zone (Made-Att)': 'red_zone_att_home', 
                            'Penalties': 'penalty_home', 
                            'Turnovers': 'turnovers_home', 
                            'Interceptions thrown': 'int_thrown_home',
                            'Fumbles lost': 'fumbles_lost_home',
                            # 'Rushing 1st downs': 'first_down_rushing_home', 
                            # 'Defensive / Special Teams TDs': 'defensive_td_home', 
                            'Possession': 'possession_home'})

game_stat_temp = pd.concat([away, home], axis=1)
game_stat_temp.head()

,,first_downs_away,third_down_away,fourth_down_away,total_yards_away,passing_away,comp_att_away,yards_per_pass_away,int_thrown_away,rushing_away,...,yards_per_pass_home,int_thrown_home,rushing_home,rushing_att_home,yards_per_rush_home,penalty_home,turnovers_home,fumbles_lost_home,int_thrown_home,possession_home
0,,8,3-14,1-1,166,72,8-22,3.3,1,94,...,10.2,2,344,44,7.8,7-44,2,0,2,31:07


going to roll all this up into a function to make the code read much easier

### loop - stats

In [2]:
games_file_path = 'data/data_crawling/table_games/2022/games.csv'
game_id_df_no_dups = pd.read_csv(games_file_path, index_col = 0)
game_id_df_no_dups.head()

,id,url,away_rank,away_team_name,away_record,home_rank,home_team_name,home_record
0,401426532,https://www.espn.com/college-football/scoreboard/_/week/1/year/2022/seasontype/2/group/80,NaN,Austin Peay,(0-10-1Away),NaN,Western Kentucky,(1-01-0Home)
1,401405059,https://www.espn.com/college-football/scoreboard/_/week/1/year/2022/seasontype/2/group/80,NaN,Nebraska,(0-10-1Big Ten),NaN,Northwestern,(1-01-0Big Ten)
2,401413256,https://www.espn.com/college-football/scoreboard/_/week/1/year/2022/seasontype/2/group/80,NaN,Idaho State,(0-10-1Away),NaN,UNLV,(1-01-0Home)
3,401404146,https://www.espn.com/college-football/scoreboard/_/week/1/year/2022/seasontype/2/group/80,NaN,UConn,(0-10-1Away),NaN,Utah State,(1-01-0Home)
4,401405058,https://www.espn.com/college-football/scoreboard/_/week/1/year/2022/seasontype/2/group/80,NaN,Wyoming,(0-10-1Away),NaN,Illinois,(1-01-0Home)


In [3]:
url_temp = 'https://www.espn.com/college-football/matchup/_/gameId/401405059'

# call game_stat_parser function from crawling helper
game_stat_temp = game_stat_parser(url_temp)

In [4]:
game_stat_temp

,first_downs_away,third_down_away,fourth_down_away,total_yards_away,passing_away,comp_att_away,yards_per_pass_away,rushing_away,rushing_att_away,yards_per_rush_away,...,comp_att_home,yards_per_pass_home,rushing_home,rushing_att_home,yards_per_rush_home,penalty_home,turnovers_home,int_thrown_home,fumbles_lost_home,possession_home
0,23,9-16,0-0,465,355,25-42,8.5,110,31,3.5,...,27-38,8.3,214,47,4.6,8-65,1,0,1,34:14


In [5]:
# loop through game_id_df_no_dups dataframe
# for each row, scrape the game stats
# append to a dataframe

master_stat_df = pd.DataFrame()

## full df
for index, row in game_id_df_no_dups.iterrows():

# # test with first 5 rows
# for index, row in game_id_df_no_dups[:5].iterrows():

    # keep game id as a variable
    game_id = str(row['id'])
    print(game_id)
    url_temp = f'https://www.espn.com/college-football/matchup/_/gameId/{game_id}'

    # call game_stat_parser function from crawling helper
    game_stat_temp = game_stat_parser(url_temp)

    # add game_id as a column
    game_stat_temp['game_id'] = game_id

    master_stat_df = pd.concat([master_stat_df, game_stat_temp], axis=0)
    master_stat_df = master_stat_df.reset_index(drop = True)

    # ensure there are no duplciate columns
    master_stat_df = master_stat_df.loc[:, ~master_stat_df.columns.duplicated()]

401426532
401405059
401413256
401404146
401405058
401411090
401426530
401411091
401409235
401426531
401403853
401416569
401411092
401411093
401416568
401403864
401415610
401426533
401441990
401405064
401416592
401426534
401426537
401405060
401403965
401413224
401415213
401416591
401416590
401426330
401426535
401411094
401404052
401405061
401403966
401403854
401404124
401403858
401403865
401403857
401405067
401426541
401424416
401411096
401426540
401411097
401405065
401403855
401403859
401403860
401426542
401415612
401404147
401405062
401405066
401405069
401411095
401415611
401411100
401415209
401404057
401403971
401403967
401405063
401415015
401415211
401403968
401421116
401415212
401420777
401426536
401426538
401403856
401403866
401404056
401404134
401404157
401414778
401421121
401425418
401403861
401403863
401426539
401403970
401404055
401404059
401411099
401415210
401403974
401403969
401403973
401426543
401403867
401411101
401413213
401422922
401423448
401418776
401421137
401428480


In [7]:
game_stat_temp.columns
# game_stat_temp

Index(['first_downs_away', 'third_down_away', 'fourth_down_away',
       'total_yards_away', 'passing_away', 'comp_att_away',
       'yards_per_pass_away', 'rushing_away', 'rushing_att_away',
       'yards_per_rush_away', 'penalty_away', 'turnovers_away',
       'int_thrown_away', 'int_thrown_away', 'fumbles_lost_away',
       'possession_away', 'first_downs_home', 'third_down_home',
       'fourth_down_home', 'total_yards_home', 'passing_home', 'comp_att_home',
       'yards_per_pass_home', 'rushing_home', 'rushing_att_home',
       'yards_per_rush_home', 'penalty_home', 'turnovers_home',
       'int_thrown_home', 'int_thrown_home', 'fumbles_lost_home',
       'possession_home', 'game_id'],
      dtype='object')

In [12]:
master_stat_df.columns

Index(['first_downs_away', 'third_down_away', 'fourth_down_away',
       'total_yards_away', 'passing_away', 'comp_att_away',
       'yards_per_pass_away', 'rushing_away', 'rushing_att_away',
       'yards_per_rush_away', 'penalty_away', 'turnovers_away',
       'int_thrown_away', 'fumbles_lost_away', 'possession_away',
       'first_downs_home', 'third_down_home', 'fourth_down_home',
       'total_yards_home', 'passing_home', 'comp_att_home',
       'yards_per_pass_home', 'rushing_home', 'rushing_att_home',
       'yards_per_rush_home', 'penalty_home', 'turnovers_home',
       'int_thrown_home', 'fumbles_lost_home', 'possession_home', 'game_id'],
      dtype='object')

In [6]:
# master_stat_df.columns
master_stat_df.to_csv('test_results.csv', index = False)

In [8]:
master_stat_df.head()

,first_downs_away,third_down_away,fourth_down_away,total_yards_away,passing_away,comp_att_away,yards_per_pass_away,rushing_away,rushing_att_away,yards_per_rush_away,...,yards_per_pass_home,rushing_home,rushing_att_home,yards_per_rush_home,penalty_home,turnovers_home,int_thrown_home,fumbles_lost_home,possession_home,game_id
0,19,7-18,2-2,319,156,15-21,7.4,163,48,3.4,...,8.4,107,32,3.3,8-86,1,1,0,27:31,401426532
1,23,9-16,0-0,465,355,25-42,8.5,110,31,3.5,...,8.3,214,47,4.6,8-65,1,0,1,34:14,401405059
2,13,5-17,4-4,241,191,14-27,7.1,50,40,1.3,...,11.6,149,33,4.5,7-65,0,0,0,30:24,401413256
3,21,6-15,1-1,364,119,13-33,3.6,245,39,6.3,...,9.1,261,54,4.8,6-60,2,0,2,31:43,401404146
4,10,1-12,0-1,212,30,5-20,1.5,182,31,5.9,...,5.4,258,41,6.3,7-80,0,0,0,36:36,401405058


In [9]:
# find where all values are 0
master_stat_df[(master_stat_df == 0).all(axis=1)]

,first_downs_away,third_down_away,fourth_down_away,total_yards_away,passing_away,comp_att_away,yards_per_pass_away,rushing_away,rushing_att_away,yards_per_rush_away,...,yards_per_pass_home,rushing_home,rushing_att_home,yards_per_rush_home,penalty_home,turnovers_home,int_thrown_home,fumbles_lost_home,possession_home,game_id


In [10]:
# check for duplciates using id column
print("Total Stats Scraped", master_stat_df.shape)

# save file to the following path - data/data_crawling/table_games/2023/games.csv
games_stat_file_path = 'data/data_crawling/table_stats/2022/game_stats.csv'

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(games_stat_file_path), exist_ok=True)

master_stat_df.to_csv(games_stat_file_path, index = False)

Total Stats Scraped (1556, 31)
